## Image conversion
Version 2021-01-04

### Load libraries

In [ ]:
import os
import pathlib
import time
import datetime
import random
import string
import numpy as np
from cv2 import cv2

# pip install scikit-image (requires version 0.17.1 or higher)
from skimage import exposure 
from skimage.filters import unsharp_mask

### Settings
Choose the desired directories and options

In [ ]:
inputdir = pathlib.Path('PATH_TO_INPUT_FOLDER') # Directory and subdirectories with IMG files
outputdir = pathlib.Path('PATH_TO_OUTPUT_FOLDER') # Output directory
filetypes = set(['.tif', '.tiff', '.png', '.jpg', '.jpeg', '.bmp']) # Define filetypes to include
output_filetype = '.png' # Filetype for output
jpg_compression = '30' # JPG compression level 
outputbitdepth = 8 # Set output bit depth
sharpen = False # Sharpen image
convert_grayscale = False # Convert to grayscale
equalize = True # CLAHE contrast enhancement
intensity_crop = 0.05 # Set rescale intensity crop 
filename_random = False # Generate random file name
relative = False # Keep folder structure if True
overwrite = False # Overwrite images

### Main code
Do not change below this line

In [ ]:
if inputdir == outputdir or not os.path.exists(inputdir):
    print('Error (Directory error): Please check folder paths.')
    exit()
    
# Generate random string (to replace filename)
def get_random_alphaNumeric_string(stringLength=8):
    lettersAndDigits = string.ascii_letters + string.digits
    return ''.join((random.choice(lettersAndDigits) for i in range(stringLength)))

# Define main function
def main():
    for paths, _, files in os.walk(os.path.normpath(inputdir), topdown=True): # Walk through subfolders
        for i, file in enumerate(files, start=1):
            time_index = str(time.strftime("%H:%M:%S", time.localtime()))+' ('+str(i).zfill(8)+')'
            try:               
                filepath = os.path.join(paths, file) # Path to input file
                reldir = os.path.relpath(paths, inputdir) # Relative path to the inputdir
                if relative == True: 
                    outputpath = os.path.normpath(os.path.join(outputdir, reldir)) # Create output path
                else: 
                    outputpath = os.path.normpath(outputdir)
                if filename_random == True: 
                    filename = get_random_alphaNumeric_string(32)
                else: 
                    filename = file
                if ('.' not in output_filetype): 
                    output_filetype_corr = '.'+output_filetype
                else: 
                    output_filetype_corr = output_filetype
                outputfile = os.path.normpath(pathlib.Path(os.path.join(outputpath, filename)).with_suffix(output_filetype_corr)) # Define outputfile and extension
                if overwrite == False and os.path.isfile(outputfile) == True: 
                    print('SKIPPED (File exists), '+filepath+' - '+time_index)
                    continue
                else:
                    if any(x in filepath.lower() for x in filetypes) == True:
                        if not os.path.exists(outputpath): 
                            os.makedirs(outputpath)
                        if convert_grayscale == True: 
                            img = cv2.imread(filepath, 0)
                        else: 
                            img = cv2.imread(filepath, -cv2.IMREAD_ANYDEPTH)
                            pass
                        if img.dtype == 'uint16': 
                            img = (img / 65535.0).astype(np.float64)
                        elif img.dtype == 'uint8': 
                            img = (img / 255.0).astype(np.float64)
                        else: 
                            print('ERROR (Input bit depth not supported) ,'+filepath+' - '+time_index)
                            continue
                        if equalize == True:
                            img = exposure.rescale_intensity(img, in_range=(np.percentile(img, intensity_crop), np.percentile(img, (100-intensity_crop))))
                            img = exposure.equalize_adapthist(img)
                        if sharpen == True: 
                            img = unsharp_mask(img, radius=1, amount=1)
                        if outputbitdepth == 8: 
                            img = cv2.normalize(img, dst=None, alpha=0, beta=int((pow(2, outputbitdepth))-1), norm_type=cv2.NORM_MINMAX).astype(np.uint8)
                        elif outputbitdepth == 16: 
                            img = cv2.normalize(img, dst=None, alpha=0, beta=int((pow(2, outputbitdepth))-1), norm_type=cv2.NORM_MINMAX).astype(np.uint16)
                        else: 
                            print('ERROR (Output bit depth not supported) ,'+filepath+' - '+time_index)
                            continue
                        if (('jpg' or 'jpeg') in output_filetype.lower()) and jpg_compression:
                            cv2.imwrite(outputfile, img, [cv2.IMWRITE_JPEG_QUALITY, int(jpg_compression)])
                            print('SUCCESS (Conversion succeeded), '+filepath+' - '+time_index)
                        else:
                            cv2.imwrite(outputfile, img)
                            print('SUCCESS (Conversion succeeded), '+filepath+' - '+time_index)
            except:
                print('ERROR (Conversion failed), '+filepath+' - '+time_index)                
    else:
        pass
   
# Call for main function
if __name__ == "__main__":  
    main()